In [1]:
import pandas as pd
import plotly.express as px

In [2]:
spacex_df = pd.read_csv("spacex_launch_dash.csv")
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [7]:
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()
print(f"Min: {min_payload}\nMax: {max_payload}")

Min: 0.0
Max: 9600.0


In [8]:
site_options_arr = ["All sites"]
for site in spacex_df['Launch Site'].unique():
    site_options_arr.append(site)

site_options_arr

['All sites', 'CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-40']

In [36]:
data = spacex_df.groupby('Launch Site')['class'].mean() # All Sites
data 

Launch Site
CCAFS LC-40     0.269231
CCAFS SLC-40    0.428571
KSC LC-39A      0.769231
VAFB SLC-4E     0.400000
Name: class, dtype: float64

In [65]:
for site in spacex_df['Launch Site'].unique():
    sucess = len(spacex_df[(spacex_df['Launch Site']==site)&(spacex_df['class']==1)])
    fail = len(spacex_df[(spacex_df['Launch Site']==site)&(spacex_df['class']==0)])
    print(f"{site} [Sucess: {sucess}, Fail: {fail}], Sucess rate: {sucess/(sucess+fail)}")

CCAFS LC-40 [Sucess: 7, Fail: 19], Sucess rate: 0.2692307692307692
VAFB SLC-4E [Sucess: 4, Fail: 6], Sucess rate: 0.4
KSC LC-39A [Sucess: 10, Fail: 3], Sucess rate: 0.7692307692307693
CCAFS SLC-40 [Sucess: 3, Fail: 4], Sucess rate: 0.42857142857142855


In [108]:
site_selected = site_options_arr[0]
if site_selected == 'All sites':
    data = spacex_df.groupby('Launch Site')['class'].mean() # All sites
else:
    data = spacex_df[spacex_df['Launch Site']==site_selected]['class'].value_counts() # Selected site
data

Launch Site
CCAFS LC-40     0.269231
CCAFS SLC-40    0.428571
KSC LC-39A      0.769231
VAFB SLC-4E     0.400000
Name: class, dtype: float64

In [83]:
fig = px.pie(data, values=data.values, names=data.index)
fig.show()

/Users/jairohotuada/Library/Python/3.9/lib/python/site-packages/plotly/express/_core.py:1976: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [139]:
site_selected = site_options_arr[0]
min_payload = 6000
max_payload = 10000
if site_selected == 'All sites':
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) & (spacex_df['Payload Mass (kg)'] <= max_payload)][['Launch Site', 'Payload Mass (kg)', 'class']]
else:
    filtered_df = spacex_df[(spacex_df['Launch Site'] == site_selected) &
                     (spacex_df['Payload Mass (kg)'] >= min_payload) & (spacex_df['Payload Mass (kg)'] <= max_payload)][['Launch Site', 'Payload Mass (kg)', 'class']]

filtered_df.head()

,Launch Site,Payload Mass (kg),class
28,VAFB SLC-4E,9600.0,1
29,VAFB SLC-4E,9600.0,1
31,VAFB SLC-4E,9600.0,1
32,VAFB SLC-4E,9600.0,0
34,VAFB SLC-4E,9600.0,0


In [137]:
filtered_df['class'].value_counts()

class
0    4
Name: count, dtype: int64

In [140]:
success_num = filtered_df['class'].value_counts().get(1, 0)
fail_count = filtered_df['class'].value_counts().get(0, 0)
sucess_rate = success_num/(success_num+fail_count)
print(filtered_df['class'].value_counts())
print("{:.2f}%".format(sucess_rate*100))


class
0    6
1    3
Name: count, dtype: int64
33.33%


In [113]:
fig = px.scatter(filtered_df, x="Payload Mass (kg)", y="class", color="Launch Site", 
                 title="Correlation between Payload and Sucess for all Sites")

fig.show()

<b>1. Which site has the largest successful launches? </b>
- KSC LC-39A
- 10 sucessfull launches
- All sucesses are coming from flights with Payload less than 5,500 kg


<b>2. Which site has the highest launch success rate?</b>
- KSC LC-39A
- 77% Sucess rate
- All sucesses are coming from flights with Payload less than 5,500 kg

<b>3. Which payload range(s) has the highest launch success rate?</b>
- 2k to 4k (61.90%)

<b>4. Which payload range(s) has the lowest launch success rate?</b>
- 6 to 8k (0%)

<b>5. Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate? </b>
- B5 with 100% sucess rate. However it only launched once
- FT next with 67% scuess rate. Sucess: 16, Fail: 8

In [157]:
for booster_cat in spacex_df['Booster Version Category'].unique():
    sucess = len(spacex_df[(spacex_df['Booster Version Category']==booster_cat)&(spacex_df['class']==1)])
    fail = len(spacex_df[(spacex_df['Booster Version Category']==booster_cat)&(spacex_df['class']==0)])
    print(f"{booster_cat} [Sucess: {sucess}, Fail: {fail}], Sucess rate: {sucess/(sucess+fail)}")

v1.0 [Sucess: 0, Fail: 5], Sucess rate: 0.0
v1.1 [Sucess: 1, Fail: 14], Sucess rate: 0.06666666666666667
FT [Sucess: 16, Fail: 8], Sucess rate: 0.6666666666666666
B4 [Sucess: 6, Fail: 5], Sucess rate: 0.5454545454545454
B5 [Sucess: 1, Fail: 0], Sucess rate: 1.0


In [156]:
spacex_df.groupby('Booster Version Category')['class'].mean()

Booster Version Category
B4      0.545455
B5      1.000000
FT      0.666667
v1.0    0.000000
v1.1    0.066667
Name: class, dtype: float64